In [1]:
# 반드시 첫 번째 셀에서 백엔드 설정
import os
os.environ['KERAS_BACKEND'] = 'torch'
# TensorFlow 완전 차단
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# PyTorch MPS 확인
import torch
if torch.backends.mps.is_available():
    print("✅ MPS (Apple Silicon GPU) available")
    torch.mps.set_per_process_memory_fraction(0.8)  # GPU 메모리 제한
else:
    print("❌ MPS not available, using CPU")

# Keras import (백엔드 확인)
import keras
print(f"🎯 Keras backend: {keras.backend.backend()}")

# TensorFlow는 import하지 않음!
import keras_hub


✅ MPS (Apple Silicon GPU) available
🎯 Keras backend: torch


In [2]:
# 환경변수 로드만
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [4]:
# PyTorch 백엔드로 Gemma 모델 로딩
gemma = keras_hub.models.GemmaCausalLM.from_preset('gemma_2b_en')

print("Gemma model loaded with PyTorch backend")
gemma.summary()  # 메모리 절약을 위해 주석 처리

Gemma model loaded with PyTorch backend


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [4]:
sampler = keras_hub.samplers.TopPSampler(p=0.8, seed=42)

gemma.compile(sampler=sampler)

# PyTorch 백엔드로 텍스트 생성 (MPS GPU 자동 사용)
text = gemma.generate('stay hungry, stay', max_length=100)

print(text)

stay hungry, stay foolish

<h1>#stayhungrystayfoolish</h1>

<strong>What does this mean?</strong>

This meme is a play on words. "Stay hungry" means to have an unending desire to learn, grow, and succeed. "Stay foolish" means to continue to explore and experiment in an effort to discover the truth.

The "not even wrong" meme comes from the popular show "The Big Bang Theory." In one episode, Sheldon Cooper (played by Jim Parsons)


In [5]:
text = gemma.generate('봄이 오면', max_length=100)

print(text)

봄이 오면 <strong>온도가 올라가서 얼마나 othermal sensing를 할 수 있는가</strong>의 문제가 있으니, <strong>본인이 연구를 하기 위한 동기가 있으면 연구를 하면 좋습니다</strong>. 또한, <strong>임상이나 산업적 문제에서 떨어져버리는 경우가 많습니다</strong>. 그럼에도 대부분의 학생들은 <strong>이 문제가 연구를 하는데


In [8]:
from transformers import pipeline, set_seed

gemma_pipe = pipeline('text-generation', model='beomi/gemma-ko-2b')

set_seed(42)

text = gemma_pipe('봄이 오면', max_length=100, truncation=True)
print(text[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps:0


봄이 오면서 겨울이 빨리 지나가는 것 같아요. 겨울이 지나고 봄이 오면서 봄꽃들이 피기 시작했어요. 봄꽃들이 피면서 봄이 오는 것 같아요. 봄이 오면서 봄꽃들이 피기 시작했어요. 봄꽃들이 피면서 봄이 오는 것 같아요. 봄이


In [5]:
gemma = keras_hub.models.GemmaCausalLM.from_preset('gemma2_2b_en')

sampler = keras_hub.samplers.TopPSampler(p=0.8, seed=42)

gemma.compile(sampler=sampler)

text = gemma.generate('stay hungry, stay', max_length=100)

print(text)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


stay hungry, stay foolish.

i'm excited for 2019! i'm looking forward to being more consistent with my workouts, my food, and my life.

thank you for all of the love and support from the past year, 2019 will be better and brighter! #grateful

Happy Holidays!
Merry Christmas!
I hope everyone has an amazing time with friends and family!
May your holidays be filled with love, laughter,


In [4]:
#gemma = keras_hub.models.Gemma3CausalLM.from_preset('gemma3_12b')
gemma = keras_hub.models.Gemma3CausalLM.from_preset('gemma3_4b')

sampler = keras_hub.samplers.TopPSampler(p=0.8, seed=42)

gemma.compile(sampler=sampler)

text = gemma.generate('stay hungry, stay', max_length=100)
print(text)

# gemma.summary()


stay hungry, stay foolish

I have been trying to get the past few months to go back to the basics of training again. I have to remind myself that my body is still healing from my surgeries and that recovery takes time. I need to continue to move slow and do what I can without pushing it too hard and too fast. I have to learn to accept that sometimes I need to go back to what I know I can do and what I can do well. I can’t do


In [ ]:
gemma = keras_hub.models.Gemma3CausalLM.from_preset('gemma3_270m')

sampler = keras_hub.samplers.TopPSampler(p=0.8, seed=42)

gemma.compile(sampler=sampler)

text = gemma.generate('stay hungry, stay', max_length=20)
print(text)

# gemma.summary()

stay hungry, stay hungry<start_of_image><start_of_image>　　北京时间7月27
